In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksstorage222.dfs.core.windows.net/products")
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()

### **Function**

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.discount_func(dis_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN dis_price * 0.9

In [0]:
%sql
select product_id, price, databricks_catalog.bronze.discount_func(price) as discounted_price 
from products

In [0]:
df = df.withColumn("discount_price",expr("databricks_catalog.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.upper_func(up_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
    return up_brand.upper()
$$

In [0]:
%sql
select product_id, brand, databricks_catalog.bronze.upper_func(brand) as BRAND 
from products

In [0]:
df.write.format("delta").mode("overwrite").option("path","abfss://silver@databricksstorage222.dfs.core.windows.net/products").save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@databricksstorage222.dfs.core.windows.net/products'